In [1]:
from rake_nltk import Rake
import nltk
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt


In [2]:
def getKeyWords(text):
    r = Rake(min_length=1, max_length=1, language="english")
    r.extract_keywords_from_text(text)

    temp = r.get_ranked_phrases()

    if len(temp) > 2:
        temp = temp[:3]
    elif len(temp) > 1:
        temp = temp[:2]
        temp.append("")
    elif len(temp) > 0:
        temp = temp[:1]
        temp.append("")
        temp.append("")
    return temp

In [3]:
def getDirectory(dirName, mDF):
    for fileName in os.listdir(dirName):
        fileNameS = fileName.split("_")
        movid = fileNameS[0]
        rating = fileNameS[1].replace(".txt", "")
        
        with open(dirName + "\\" + fileName, "r", encoding ="utf-8") as myfile:
            text = myfile.read()
            
            
        keyWords = getKeyWords(text)
        keyWords.insert(0, rating)
        keyWords.insert(0, movid)
        
        #print(keyWords)
        columns = [
            "ID",
            "rating",
            "keyword1",
            "keyword2",
            "keyword3"
        ]
        
        movieDF = pd.DataFrame(pd.Series(keyWords), index = columns)
        
        
        mDF.append(movieDF)

In [4]:
def train(trainDF):
    targetY = trainDF["rating"]
    trainX = trainDF.drop("rating",  axis = 1).drop("ID",  axis = 1)
    lm = LinearRegression()
    lm.fit(trainX, targetY)

In [ ]:
def main():
    columns = [
        "ID",
        "rating",
        "keyword1",
        "keyword2",
        "keyword3"
    ]

    mDF = pd.DataFrame(columns = columns)
    home_dir = os.path.abspath('').replace("scripts", "")
    negTrain = home_dir + "\\data\\aclImdb\\train\\neg"
    posTrain = home_dir + "\\data\\aclImdb\\train\\pos"
    
    getDirectory(negTrain, mDF)
    getDirectory(posTrain, mDF)
    train(mDF)
    
    
    tDF = pd.DataFrame(columns = columns)
    negTest = home_dir + "\\data\\aclImdb\\test\\neg"
    posTest = home_dir + "\\data\\aclImdb\\test\\pos"
    getDirectory(negTest, tDF)
    getDirectory(posTest, tDF)
    
    Y_actual = tDF["rating"]
    X_test = tDF.drop("rating",  axis = 1).drop("ID",  axis = 1)
    
    Y_pred = lm.predict(X_test)
    #print(mDF)
    plt.scatter(Y_actual, Y_pred)
    plt.show()
main()